In [170]:
# read from json file
import json 
with open('data.jsonl') as jsonl_file:
    lines = jsonl_file.readlines()
jsonS = [json.loads(line) for line in lines]
print(len(jsonS))

3702


In [340]:
with open('annotations2.jsonl') as jsonl_file:
    lines = jsonl_file.readlines()
annot = [json.loads(line) for line in lines]
#print(annot[0])

In [400]:
def getLabel(listDict,idxEnd): 
    # note: this function assumes annot[k]['tokens'] are laid out in ascending order in which they appear in text
    # this is not the case for the new 1 char tokens we added to correct labelling inconsistencies
    # but since they are all classifies as "O" anyway it doesn't matter 
    for i in range(0,len(listDict)):
        auxDict=listDict[i]
        a_s, a_e = auxDict['start'], auxDict['end']
        if idxEnd>=a_s: # i.e. in any case stop when idxStart < a_s (as dict tokens are ordered ASC in list; i.e. by order they appear in text)
            if (idxEnd>=a_s and idxEnd<=a_e): 
                return auxDict['label'] 
            elif i==len(listDict)-1: #case where idxStart > 'end' key of last dict in list
                if idxEnd>a_e:
                    return 'O'
        else: return 'O' 
# test function        
#a=[{'start':0, 'end':3, 'label': 'ORT'},{'start':5, 'end':8, 'label': 'ORT2'},{'start':10, 'end':13, 'label': 'ORT3'}]
#b=3
#print(getLabel(a,b))

In [401]:
# now use above code and loop through all items of annot list:
# addLeading=1 for "Yes" (i.e. add leading "B-","I-" to annot); 0 for "No" (i.e. add labels to annot simply as they are)
addLeading = 1
    
for j in range(0,len(annot)):
    a = annot[j]
    # select list of dict of tokens w/ annnotations and add column w/ no. of words to each dict:
    b = a['spans']
    # add noWords to b dict. note: b is list of dicts w/ annotations; tokens not on this list don't have annotations
    if b!=[]: #i.e. only try to add annotations to tokens if there are annotations to begin with
        for i in range(0,len(b)): 
            b[i]['noWords']=b[i]['token_end']-b[i]['token_start']+1
        #print(b)

        # select dict of tokens
        c = a['tokens']
        # add label to each token
        for i in range(0,len(c)):
            c[i]['label'] = getLabel(b,c[i]['end']) # chged from 'start' to 'end'
            if addLeading == 1:
                # now break-up label into 1st occurrence (leading "B-") and subsequent occurrences (leading "I-") (only for non "O"'s)
                if c[i]['label'] != "O":
                    if i==0:
                        c[i]['label'] = "B-" + c[i]['label']
                    else: 
                        if c[i]['label'] == c[i-1]['label'][2:]: # need to remove the leading "B-" that we had already been added to c[i-1]
                            c[i]['label'] = "I-" + c[i]['label']
                        else:
                            c[i]['label'] = "B-" + c[i]['label'] 
        annot[j]['tokens'] = c

In [402]:
# now convert annotation tokens into list (sentences) of lists (tokens) format for sklearn_crfsuite.CRF
train_sents=[] 
for j in range(0,len(annot)):
    a = annot[j]['tokens']
    train_sentence = []
    for i in range(0,len(a)):
        if 'label' in a[i]: # only add element if this sample sentence has been labelled 
            token_element = (a[i]['text'],a[i]['label'])
            train_sentence.append(token_element)
    train_sents.append(train_sentence)

In [403]:
print(train_sents[0])

[('DORNBIRN', 'B-ORT'), ('In', 'O'), ('der', 'O'), ('Schulgasse', 'B-STRASSE'), ('in', 'O'), ('Dornbirn', 'B-ORT'), ('hat', 'O'), ('eine', 'O'), ('71,93', 'B-FLAECHE'), ('Quadratmeter', 'O'), ('große', 'O'), ('Wohnung', 'B-IMMO_TYP'), ('für', 'O'), ('einen', 'O'), ('Quadratmeterpreis', 'O'), ('von', 'O'), ('5533,71', 'B-QMPREIS'), ('Euro', 'O'), ('den', 'O'), ('Besitzer', 'O'), ('gewechselt', 'O'), ('.', 'O'), ('Dieser', 'O'), ('beinhaltet', 'O'), ('auch', 'O'), ('einen', 'O'), ('Pkw-Abstellplatz', 'O'), ('.', 'O'), ('Käufer', 'O'), ('der', 'O'), ('Wohnung', 'O'), ('mit', 'O'), ('9,86', 'B-TERRASSENGROESSE'), ('Quadratmetern', 'O'), ('Terrasse', 'O'), ('ist', 'O'), ('die', 'O'), ('ValLiLean', 'B-KAEUFER'), ('Beteiligungs-', 'I-KAEUFER'), ('und', 'I-KAEUFER'), ('Immobilienverwaltungs', 'I-KAEUFER'), ('GmbH', 'I-KAEUFER'), ('Beim', 'O'), ('Verkäufer', 'O'), ('handelt', 'O'), ('es', 'O'), ('sich', 'O'), ('um', 'O'), ('die', 'O'), ('Karrenblick', 'B-VERKAEUFER'), ('Projekt', 'I-VERKAEUFER'

In [389]:
print(annot[0]['spans'])

[{'text': 'DORNBIRN', 'start': 0, 'end': 8, 'pattern': 2069086582, 'token_start': 0, 'token_end': 0, 'label': 'ORT', 'noWords': 1}, {'start': 16, 'end': 26, 'token_start': 3, 'token_end': 3, 'label': 'STRASSE', 'noWords': 1}, {'text': 'Dornbirn', 'start': 30, 'end': 38, 'pattern': 2069086582, 'token_start': 5, 'token_end': 5, 'label': 'ORT', 'noWords': 1}, {'start': 48, 'end': 53, 'token_start': 8, 'token_end': 8, 'label': 'FLAECHE', 'noWords': 1}, {'start': 73, 'end': 80, 'token_start': 11, 'token_end': 11, 'label': 'IMMO_TYP', 'noWords': 1}, {'start': 113, 'end': 120, 'token_start': 16, 'token_end': 16, 'label': 'QMPREIS', 'noWords': 1}, {'start': 221, 'end': 225, 'token_start': 32, 'token_end': 32, 'label': 'TERRASSENGROESSE', 'noWords': 1}, {'start': 257, 'end': 311, 'token_start': 37, 'token_end': 41, 'label': 'KAEUFER', 'noWords': 5}, {'start': 351, 'end': 375, 'token_start': 49, 'token_end': 51, 'label': 'VERKAEUFER', 'noWords': 3}, {'text': '398.040', 'start': 402, 'end': 409, 

In [390]:
b=annot[0]['spans']
c=annot[0]['tokens']
i=74
print(b)
print(c[i]['start'])
print(getLabel(b,c[i]['end']))

[{'text': 'DORNBIRN', 'start': 0, 'end': 8, 'pattern': 2069086582, 'token_start': 0, 'token_end': 0, 'label': 'ORT', 'noWords': 1}, {'start': 16, 'end': 26, 'token_start': 3, 'token_end': 3, 'label': 'STRASSE', 'noWords': 1}, {'text': 'Dornbirn', 'start': 30, 'end': 38, 'pattern': 2069086582, 'token_start': 5, 'token_end': 5, 'label': 'ORT', 'noWords': 1}, {'start': 48, 'end': 53, 'token_start': 8, 'token_end': 8, 'label': 'FLAECHE', 'noWords': 1}, {'start': 73, 'end': 80, 'token_start': 11, 'token_end': 11, 'label': 'IMMO_TYP', 'noWords': 1}, {'start': 113, 'end': 120, 'token_start': 16, 'token_end': 16, 'label': 'QMPREIS', 'noWords': 1}, {'start': 221, 'end': 225, 'token_start': 32, 'token_end': 32, 'label': 'TERRASSENGROESSE', 'noWords': 1}, {'start': 257, 'end': 311, 'token_start': 37, 'token_end': 41, 'label': 'KAEUFER', 'noWords': 5}, {'start': 351, 'end': 375, 'token_start': 49, 'token_end': 51, 'label': 'VERKAEUFER', 'noWords': 3}, {'text': '398.040', 'start': 402, 'end': 409, 

In [404]:
print(annot[0]['text'][311:312])

.


In [343]:
import re

In [100]:
aaa="350.000 Euro"
bbb="axyz"
regex = re.compile(r'(\d\.?)?(\d{3}\.?\d{3})? Euro')
matches = regex.finditer(aaa)
for match in matches:
    print(match)
print(bool(re.match(r'(\d\.?)?(\d{3}\.?\d{3})',aaa)))

<re.Match object; span=(0, 12), match='350.000 Euro'>
True


In [344]:
import pandas as pd
df = pd.read_csv('townsVoralberg.csv',names=['towns'])  

In [337]:
print(len(df['towns']))

105


In [345]:
x = lambda a: str(a).lower()

In [150]:
df.head()
aaa="hochst"
print(aaa in str(df['towns']).lower())
a = map(x,df['towns'])
print(a)
print(aaa in a)

False
False


In [346]:
def word2features(sent, i):
    word = sent[i][0]
    word_l = word.lower()
    towns = map(x,df['towns'])
    #postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word_l,
        'word[-3:]': word[-3:], # 3 char ending of word (suffix)
        'word[-2:]': word[-2:], # 2 char ending of word (suffix)
        'word.isupper()': word.isupper(), # all letters are caps
        'word.istitle()': word.istitle(), # 1st letter is a capital letter
        'word.isdigit()': word.isdigit(), # is digit?
        'word.isPrice()': bool(re.match(r'(\d\.?)?(\d{3}\.?\d{3})',word)),
        'word.isTown()': word_l in towns,
        #'postag': postag, # don't have PoS data
        #'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0] # enriches feature function w/ immediately preceding word
        word1_l = word1.lower()
        #postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1_l,
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:word.isPrice()': bool(re.match(r'(\d\.?)?(\d{3}\.?\d{3})',word1)),
            '-1.word.isTown()': word1_l in towns,
            #'-1:postag': postag1,
            #'-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0] # ... as well as w/ immediate sucessor word 
        word1_l = word1.lower()
        #postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1_l,
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1word.isPrice()': bool(re.match(r'(\d\.?)?(\d{3}\.?\d{3})',word1)),
            '+1.word.isTown()': word1_l in towns,
            #'+1:postag': postag1,
            #'+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
   # return [label for token, postag, label in sent]
    return [label for token, label in sent]

def sent2tokens(sent):
   # return [token for token, postag, label in sent]
    return [token for token, label in sent]

In [185]:
sent2features(train_sents[0])[2]

{'bias': 1.0,
 'word.lower()': 'der',
 'word[-3:]': 'der',
 'word[-2:]': 'er',
 'word.isupper()': False,
 'word.istitle()': False,
 'word.isdigit()': False,
 'word.isPrice()': False,
 'word.isTown()': False,
 '-1:word.lower()': 'in',
 '-1:word.istitle()': True,
 '-1:word.isupper()': False,
 '-1:word.isPrice()': False,
 '-1.word.isTown()': False,
 '+1:word.lower()': 'schulgasse',
 '+1:word.istitle()': True,
 '+1:word.isupper()': False,
 '+1word.isPrice()': False,
 '+1.word.isTown()': False}

In [405]:
X_train = [sent2features(s) for s in train_sents] # both are 140 x # of tokens in each example
y_train = [sent2labels(s) for s in train_sents]

In [104]:
print(len(X_train[1]))
print(len(y_train[1]))

72
72


In [406]:
import sklearn
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
import scipy.stats
import eli5
%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train[:120], y_train[:120])

Wall time: 0 ns


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [407]:
eli5.show_weights(crf, top=30)

From \ To,O,B-DATUM_VERBUECHERUNG,I-DATUM_VERBUECHERUNG,B-DATUM_VERTRAG,I-DATUM_VERTRAG,B-FLAECHE,I-FLAECHE,B-GESAMTPREIS,I-GESAMTPREIS,B-IMMO_TYP,I-IMMO_TYP,B-KAEUFER,I-KAEUFER,B-ORT,B-QMPREIS,I-QMPREIS,B-STRASSE,I-STRASSE,B-TERRASSENGROESSE,B-VERKAEUFER,I-VERKAEUFER
O,1.948,2.402,-1.656,1.743,-1.672,1.775,-0.895,1.358,-1.355,1.894,-3.122,2.177,-2.089,1.584,1.105,-0.747,1.702,-2.383,0.592,1.917,-2.051
B-DATUM_VERBUECHERUNG,-0.923,-0.631,3.953,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.041,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I-DATUM_VERBUECHERUNG,-1.037,0.0,2.371,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-DATUM_VERTRAG,1.018,-0.83,-0.556,-0.889,4.146,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I-DATUM_VERTRAG,0.161,0.0,0.0,0.0,2.174,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-FLAECHE,0.744,0.0,0.0,0.0,0.0,0.0,1.61,0.0,0.0,-0.204,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.132
I-FLAECHE,0.048,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-GESAMTPREIS,0.405,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.371,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I-GESAMTPREIS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.102,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-IMMO_TYP,-0.044,0.0,-0.257,0.0,-0.128,-0.285,0.0,0.487,0.0,-0.305,2.853,0.0,-0.12,0.0,-0.154,0.0,0.171,-0.153,0.0,0.0,-0.056


In [408]:
labels = list(crf.classes_)
labels.remove('O')
labels

['B-ORT',
 'B-STRASSE',
 'B-FLAECHE',
 'B-IMMO_TYP',
 'B-QMPREIS',
 'B-TERRASSENGROESSE',
 'B-KAEUFER',
 'I-KAEUFER',
 'B-VERKAEUFER',
 'I-VERKAEUFER',
 'B-GESAMTPREIS',
 'B-DATUM_VERTRAG',
 'B-DATUM_VERBUECHERUNG',
 'I-DATUM_VERBUECHERUNG',
 'I-IMMO_TYP',
 'I-STRASSE',
 'I-DATUM_VERTRAG',
 'I-GESAMTPREIS',
 'I-QMPREIS',
 'I-FLAECHE']

In [409]:
X_test = X_train[120:]
y_test = y_train[120:]

In [410]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred,
                      average='weighted', labels=labels)

0.9652749006112449

In [411]:
# group B and I results
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

                       precision    recall  f1-score   support

B-DATUM_VERBUECHERUNG      1.000     1.000     1.000        14
I-DATUM_VERBUECHERUNG      1.000     1.000     1.000        14
      B-DATUM_VERTRAG      1.000     1.000     1.000        15
      I-DATUM_VERTRAG      1.000     1.000     1.000         4
            B-FLAECHE      0.944     1.000     0.971        17
            I-FLAECHE      0.000     0.000     0.000         0
        B-GESAMTPREIS      1.000     1.000     1.000        13
        I-GESAMTPREIS      0.000     0.000     0.000         0
           B-IMMO_TYP      0.947     0.857     0.900        21
           I-IMMO_TYP      0.000     0.000     0.000         0
            B-KAEUFER      0.900     1.000     0.947         9
            I-KAEUFER      1.000     1.000     1.000         8
                B-ORT      1.000     1.000     1.000        33
            B-QMPREIS      1.000     1.000     1.000        11
            I-QMPREIS      0.000     0.000     0.000  

C:\Users\vasco\anaconda3\envs\xcs224u\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass labels=['B-DATUM_VERBUECHERUNG', 'I-DATUM_VERBUECHERUNG', 'B-DATUM_VERTRAG', 'I-DATUM_VERTRAG', 'B-FLAECHE', 'I-FLAECHE', 'B-GESAMTPREIS', 'I-GESAMTPREIS', 'B-IMMO_TYP', 'I-IMMO_TYP', 'B-KAEUFER', 'I-KAEUFER', 'B-ORT', 'B-QMPREIS', 'I-QMPREIS', 'B-STRASSE', 'I-STRASSE', 'B-TERRASSENGROESSE', 'B-VERKAEUFER', 'I-VERKAEUFER'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


In [167]:
from collections import Counter 
# summary counts
# first unfold y_train
y_train_all = [item for sublist in y_train[120:] for item in sublist]
y_train_all_count = Counter(y_train_all)
print(y_train_all_count)
print(len(y_train_all))
print(len(y_train))

Counter({'O': 920, 'B-ORT': 34, 'I-DATUM_VERTRAG': 28, 'B-IMMO_TYP': 22, 'I-VERKAEUFER': 22, 'I-KAEUFER': 18, 'B-FLAECHE': 17, 'B-VERKAEUFER': 15, 'B-DATUM_VERTRAG': 15, 'B-DATUM_VERBUECHERUNG': 15, 'I-DATUM_VERBUECHERUNG': 15, 'B-STRASSE': 14, 'B-GESAMTPREIS': 13, 'B-KAEUFER': 12, 'B-QMPREIS': 11, 'I-STRASSE': 5, 'B-TERRASSENGROESSE': 5, 'I-ORT': 1, 'I-IMMO_TYP': 1})
1183
140


In [197]:
# extract counters for all annot 
for l in range(0,len(labels)):
    out=[]
    for k in range(0,len(annot)):
        j=len(annot[k]['tokens'])
        for i in range(0,j):
            if annot[k]['spans']!=[]: # are there annot for this example?
                if annot[k]['tokens'][i]['label']==labels[l]:
                    out.append(annot[k]['tokens'][i]['text'].lower())
    out_count = Counter(out)
    print(out_count)

Counter({'dornbirn': 37, 'feldkirch': 25, ':': 20, 'hohenems': 15, 'bregenz': 12, 'götzis': 9, 'lustenau': 9, 'schruns': 8, 'rankweil': 8, 'bludenz': 7, 'höchst': 6, 'lauterach': 5, 'ludesch': 4, 'vandans': 4, 'bludesch': 4, 'hörbranz': 4, 'lochau': 4, 'frastanz': 4, 'schwarzach': 4, 'nüziders': 4, 'wolfurt': 4, 'satteins': 4, 'koblach': 3, 'schlins': 3, 'zwischenwasser': 3, 'brand': 3, 'nenzing': 3, 'lech': 2, 'göfis': 2, 'sulzberg': 2, 'klaus-weiler': 2, 'egg': 2, 'hohenweiler': 2, 'bartholomäberg': 2, 'mäder': 2, 'bürs': 2, 'riefensberg': 2, 'altach': 2, 'klösterle': 2, 'blons': 2, 'fontanella': 2, 'schröcken': 2, 'hard': 2, 'krumbach': 2, 'doren': 2, 'zürs': 1, 'gaissau': 1, 'gaißau': 1, 'bürserberg': 1, 'fußach': 1, '.': 1, 'klaus': 1})
Counter({'der': 6, 'im': 5, 'in': 4, 'walgaustraße': 3, 'radetzkystraße': 3, 'bereich': 2, 'bachgasse': 2, '9': 2, 'quellengasse': 2, '19': 2, '7': 2, 'auf': 2, 'römerstraße': 2, 'hag': 2, 'bergäcker': 2, 'sandgasse': 2, '2': 2, '35': 2, 'landstraß

In [198]:
print(labels)

['ORT', 'STRASSE', 'FLAECHE', 'IMMO_TYP', 'GESAMTPREIS', 'TERRASSENGROESSE', 'KAEUFER', 'VERKAEUFER', 'DATUM_VERTRAG', 'DATUM_VERBUECHERUNG', 'QMPREIS']


In [270]:
# extract # of tokens for each annot 
out=Counter({})
for k in range(0,len(annot)):
    j=len(annot[k]['tokens']) #74
    i=0
    if annot[k]['spans']!=[]: # are there annot for this example?
        while i < j:
            m=0 # count length of successive tokens w/ same label
            #print("k/"+str(k))
            #print("i/"+str(i))
            while (i < (j-1)) and annot[k]['tokens'][i]['label']==annot[k]['tokens'][i+1]['label']:
                 # note w/ i=j-2 in below expression can go up to i+1 = j-1 ... i.e. i can go up to 72
                m+=1
                i+=1
                #print(annot[k]['tokens'][i]['label'])
                #print(m)
            m+=1
            out += Counter({(annot[k]['tokens'][i]['label'],m): 1}) # add 1 entry for this pair of label and length
            #if (annot[k]['tokens'][i]['label'] == 'GESAMTPREIS') and (m>1): # investigating situations where # successive tokens GESAMTPREIS > 1
            #    print("k/"+str(k))
            #    print("i/"+str(i))
            i+=1
            #print(out)
                   # out.append(annot[k]['tokens'][i]['text'].lower())
print(out)

k/20
i/8
k/21
i/15
k/27
i/21
Counter({('O', 2): 281, ('ORT', 1): 217, ('O', 1): 212, ('O', 4): 192, ('FLAECHE', 1): 138, ('IMMO_TYP', 1): 135, ('O', 7): 128, ('DATUM_VERBUECHERUNG', 2): 104, ('GESAMTPREIS', 1): 97, ('DATUM_VERTRAG', 3): 94, ('QMPREIS', 1): 75, ('O', 5): 67, ('O', 6): 60, ('VERKAEUFER', 2): 57, ('STRASSE', 1): 53, ('KAEUFER', 2): 45, ('O', 3): 42, ('STRASSE', 2): 39, ('TERRASSENGROESSE', 1): 36, ('O', 8): 28, ('ORT', 2): 21, ('KAEUFER', 3): 19, ('VERKAEUFER', 3): 18, ('O', 9): 18, ('O', 13): 18, ('O', 12): 17, ('O', 10): 15, ('IMMO_TYP', 4): 14, ('O', 11): 12, ('O', 14): 10, ('O', 15): 9, ('STRASSE', 3): 9, ('KAEUFER', 4): 9, ('DATUM_VERTRAG', 2): 9, ('VERKAEUFER', 5): 8, ('IMMO_TYP', 2): 5, ('KAEUFER', 5): 4, ('DATUM_VERBUECHERUNG', 3): 4, ('VERKAEUFER', 4): 4, ('IMMO_TYP', 5): 3, ('DATUM_VERTRAG', 4): 3, ('VERKAEUFER', 1): 3, ('GESAMTPREIS', 2): 3, ('O', 18): 3, ('VERKAEUFER', 6): 2, ('STRASSE', 4): 2, ('VERKAEUFER', 8): 2, ('VERKAEUFER', 7): 2, ('O', 16): 2, ('IMMO_T

In [304]:
# find situations where token ends with "."
for l in range(0,len(labels)):
    out=[]
    for k in range(0,len(annot)):
        j=len(annot[k]['tokens'])
        for i in range(0,j):
            if annot[k]['spans']!=[]: # are there annot for this example?
                if annot[k]['tokens'][i]['label']==labels[l]:
                    if annot[k]['tokens'][i]['text'].lower()[-1:] == '.':
                        out.append(annot[k]['tokens'][i]['text'].lower())
                        #print("k/"+str(k))
    out_count = Counter(out)
    print(out_count)

Counter({'.': 1})
Counter()
Counter()
Counter({'.': 2})
Counter({'1.': 1})
Counter()
Counter({'.': 46, 'gmbh.': 22, 'mbh.': 1, 'd.e.': 1})
Counter({'.': 57, 'gmbh.': 20, '..': 1})
Counter({'.': 90, '6.': 9, '12.': 7, '3.': 7, '18.': 6, '1.': 6, '29.': 6, '2.': 6, '4.': 5, '27.': 5, '5.': 5, '23.': 5, '16.': 4, '7.': 3, '10.': 3, '28.': 3, '20.': 3, '25.': 3, '31.': 3, '2020.': 3, '19.': 2, '2019.': 2, '15.': 2, '30.': 2, '24.': 2, '21.': 2, '11.': 1, '9.': 1, '25.04.2019.': 1, '8.': 1, '13.': 1, '2012.': 1, '14.': 1, '26.': 1})
Counter({'2021.': 55, '2020.': 42, '.': 7, '2022.': 2, '2019.': 1})
Counter()


In [331]:
# find situations where last token of same label ends with "." or ":"
for l in range(0,len(labels)):
    out=[]
    for k in range(0,len(annot)):
        j=len(annot[k]['tokens'])
        for i in range(0,j-1):
            if annot[k]['spans']!=[]: # are there annot for this example?
                if annot[k]['tokens'][i]['label']==labels[l]: 
                    #if annot[k]['tokens'][i]['text'].lower()[-1:] in [',',':','.',';'] and annot[k]['tokens'][i]['label'] != annot[k]['tokens'][i+1]['label']:
                    if bool(re.match(r'\W',annot[k]['tokens'][i]['text'].lower()[-1:])) and annot[k]['tokens'][i]['label'] != annot[k]['tokens'][i+1]['label']:
                        out.append(annot[k]['tokens'][i]['text'].lower())
                        #print("k/"+str(k))
        # now add for i=j-1 (i.e. last token in example; by definition this is last token in label)
        if annot[k]['spans']!=[]: # are there annot for this example?
               # if annot[k]['tokens'][i]['label']==labels[l]:
            if bool(re.match(r'\W',annot[k]['tokens'][i]['text'].lower()[-1:])):
                if annot[k]['tokens'][j-1]['text'].lower()[-1:] in [',',':','.',';']:
                    out.append(annot[k]['tokens'][j-1]['text'].lower())
    out_count = Counter(out)
    print(out_count)

Counter({':': 20, '.': 2})
Counter({'.': 1})
Counter({'.': 1})
Counter({'.': 3, ',': 1})
Counter({'1.': 1, '.': 1})
Counter({'.': 1})
Counter({'.': 47, 'gmbh.': 22, ',': 4, 'mbh.': 1})
Counter({'.': 57, 'gmbh.': 20, ',': 7, '..': 1})
Counter({'.': 91, '2020.': 3, '2019.': 2, '25.04.2019.': 1, '2012.': 1, ',': 1})
Counter({'.': 2, ')': 1})
Counter({'.': 1})


In [318]:
word='abcd2'
bool(re.match(r'\W',word[-1:]))

False

In [327]:
word = 'abcd.'
print(word[-1:])
word[-1:] in [',',':','.',';']

.


True

In [329]:
print(annot[1])

{'text': 'FELDKIRCH Im Altenreuteweg in Feldkirch hat eine 100,67 Quadratmeter große Wohnung für einen Quadratmeterpreis von 6168,67 Euro den Besitzer gewechselt. Käufer der Wohnung mit einer 137,49 Quadratmeter großen Terrasse ist eine Privatperson. Beim Verkäufer handelt es sich um die Rüscher und Söhne Bau GmbH & Co KG.  Der Kaufpreis liegt bei 621.000 Euro. Unterzeichnet wurde der Kaufvertrag am 11. August. Die Verbücherung datiert mit September 2020.', 'meta': {'source': 'vn', 'pattern': '24, 24, 96'}, '_input_hash': -747520372, '_task_hash': -1912384731, 'spans': [{'text': 'FELDKIRCH', 'start': 0, 'end': 9, 'pattern': 122506994, 'token_start': 0, 'token_end': 0, 'label': 'ORT', 'noWords': 1}, {'start': 13, 'end': 26, 'token_start': 2, 'token_end': 2, 'label': 'STRASSE', 'noWords': 1}, {'text': 'Feldkirch', 'start': 30, 'end': 39, 'pattern': 122506994, 'token_start': 4, 'token_end': 4, 'label': 'ORT', 'noWords': 1}, {'start': 49, 'end': 55, 'token_start': 7, 'token_end': 7, 'label